# 2D Alignment

In [ ]:
using Revise
push!(LOAD_PATH, "$(homedir())/Development/")
using JSort
using Plots
using PyCall
using StatsPlots
import CSV
using StatsBase
using DataFrames

In [ ]:
# Load the preprocessed data
#events = loadlabr("sirius");
#parameters = Parameters("si28.yaml");

# Sort the datafile
#mede, medebf = makeeΔe(events, parameters);

## Old OMatrix Method

In [ ]:
function plotmat(matrix)
    G = convert(Array{Float64, 2}, matrix)
    log10transform!(G)
    pl = heatmap(G', c=:viridis, yticks=([1:length(chns);], chns))
end

function plotstrip(strips)
    G = hcat(strips...)
    G = convert(Array{Float64, 2}, G)
    log10transform!(G)
    pl = heatmap(G', c=:viridis)
end

function log10transform!(G)
    G[G .> 0] .= log10.(G[G .> 0])
    G[G .≤ 0] .= NaN
end

function log10transform(V)
    G = zeros(size(V))
    G[V .> 0] .= log10.(G[V .> 0])
    G[V .≤ 0] .= NaN
    G
end


log10transform (generic function with 1 method)

In [ ]:
heatmap(log10transform(medebf[1, 1].matrix'), color=:viridis)

In [ ]:
function plotmat(matrix)
    G = convert(Array{Float64, 2}, matrix)
    log10transform!(G)
    pl = heatmap(G', c=:viridis, yticks=([1:length(chns);], chns))
end

function plotstrip(strips)
    G = hcat(strips...)
    G = convert(Array{Float64, 2}, G)
    log10transform!(G)
    pl = heatmap(G', c=:viridis, size=(1000, 1000))
end

function log10transform!(G)
    G[G .> 0] .= log10.(G[G .> 0])
    G[G .≤ 0] .= NaN
end

function log10transform(V)
    G = V[1:end]
    G[G .> 0] .= log10.(G[G .> 0])
    G[G .≤ 0] .= NaN
    G
en

In [ ]:
@recipe f(::Type{OMatrix{Array{Int64, 2}}}, omatrix::OMatrix{Array{Int64, 2}}) = log10transform(omatrix.matrix')

In [ ]:
plot(refmat.matrix, seriestype=:image)

In [ ]:
function plotbox(edges)
    (startx, stopx), (starty, stopy) = edges
    plot!([startx, stopx, stopx, startx, startx], 
        [starty, starty, stopy, stopy, starty], color=:steelblue, legend=false)
end

function plotbox(subarray::SubArray)
    x, y = subarray.indices
    plotbox(((first(x), last(x)), (first(y), last(y))))
end

plotbox (generic function with 2 methods)

In [ ]:
edges = boxbounds(medebf[1, 1].matrix, 8)
subarrays = [arraybox(medebf[1, 1].matrix, edge) for edge in edges]
plt = heatmap(medebf[1,1], color=:viridis)
plotbox.(subarrays)
plt

In [ ]:
arrays = splitregions2d(medebf[1, 1].matrix, 20, limit=1000)
plt = heatmap(medebf[1,1], color=:viridis)
plotbox.(arrays)
plt

In [ ]:
heatmap(medebf[1, 1], color=:viridis)
heatmap!(medebf[1, 2], color=:blues)
plotbox(arrays[5])
overlap = featureoverlap2d(arrays[5], medebf[1, 2].matrix, 10)
@show overlap
@show arrays[5].indices
#heatmap(overlap)

In [ ]:
refmat = medebf[1, 1]
tarmat = medebf[1, 5]
ref, tar = featurealign2d(refmat, tarmat, 
    numregions=10, width=100, limit=1000, searchwidth=10)
arrays = splitregions2d(medebf[1, 1], 10, width=100, limit=1000);

In [ ]:
plt = heatmap(xrange(refmat), yrange(refmat), refmat, color=:viridis)
#plt = heatmap!(range(0, 1000, length=size(tarmat.matrix, 1)), yrange(tarmat), tarmat, color=:greens)
plotbox.(arrays)
for (r, t) in zip(ref, tar)
    rx, ry = r
    tx, ty = t
    scatter!([rx, tx], [ry, ty], legend=false, markershape=:dtriangle, markerstrokewidth=0,
    markersize=4)
end
plt

In [ ]:
x, y = zip([([r[1], t[1]], [r[2], t[2]]) for (r, t) in zip(ref, tar)]...)
refx, tarx = zip(x...) .|> collect .|> indextox(x, tarmat)
refy, tary = zip(y...) .|> collect
coeffx = leastsquares(tarx, refx; order=1)
coeffy = leastsquares(tary, refy, order=1)

@show coeffx
@show coeffy
@show trangex = xrange(tarmat)
@show trangey = yrange(tarmat)
@show ttrangex = coeffx[1] .+ trangex.*coeffx[2]
@show ttrangey = coeffy[1] .+ trangey.*coeffy[2]


In [ ]:
bf14 = CSV.read("sirius/medeb1f4.csv", header=["e", "Δe"]);
bf25 = CSV.read("sirius/medeb2f5.csv", header=["e", "Δe"]);

In [ ]:
b = bf14[1:1000, :];
b2 = bf25[1:1000, :];

In [ ]:
 Gadfly.plot(b, x=:e, y=:Δe, Geom.hexbin)

In [ ]:
@df bf25 Plots.plot(:e, :Δe, markerstrokewidth=0, seriestype=:histogram2d, nbins=400, 
                 color=:reds)
@df bf14 Plots.plot!(:e, :Δe, markerstrokewidth=0, seriestype=:histogram2d, nbins=400,
                color=:blues)

## New Method

Load the data

In [ ]:
reference = CSV.read("sirius/medeb1f1.csv", header=["e", "Δe"]);
target = CSV.read("sirius/medeb2f5.csv", header=["e", "Δe"]);

Bin the data

In [ ]:
function binner(edges)
    start = first(edges)
    stop = last(edges)
    dx = step(edges)
    maxi = length(edges)
    function tobin(x)::Int64
        if x < start
            1
        elseif x > stop
            maxi
        else
            trunc(Int, (x - start)/dx) + 1  
        end
    end
    tobin
end

function binhist(X, Y, xedges, yedges)
    tobinx = binner(xedges)
    tobiny = binner(yedges)
    weights = zeros((length(xedges), length(yedges)))
    for (x, y) in zip(X, Y)
        i = tobinx(x)
        j = tobiny(y)
        weights[i, j] += 1
    end
    weights
end

In [ ]:
using PyCall
pygui(true)
using PyPlot

In [ ]:
refhist = fit(Histogram, (reference[:e], reference[:Δe]), nbins=1000)
tarmat = binhist(target[:e], target[:Δe], refhist.edges...);
refmat = refhist.weights;

In [ ]:
rpx, tpx, rpy, tpy = featurealign2d(refmats, tarmats, 
    numregions=15, width=nothing, limit=1000, searchwidth=40)

@show coeffx = leastsquares(tpx, rpx; order=1)
@show coeffy = leastsquares(tpy, rpy; order=1);

# The coefficients are in index-basis. Need them in energy basis

In [ ]:
e = @. coeffx[1] + coeffx[2]*target[:e]
de = @. coeffy[1] + coeffy[2]*target[:Δe]

calibtarget = DataFrame(e=e, Δe=de);
calibmat = binhist(calibtarget[:e], calibtarget[:Δe], refhist.edges...);

In [ ]:
fig, ax = PyPlot.subplots(nrows=2)

for mat in (tarmat, refmat, calibmat)
    y = sum(mat, dims=1)'
    x = 1:length(y)
    ax[1].step(x, y)
    y = sum(mat, dims=2)
    x = 1:length(y)
    ax[2].step(x, y)
end
ax[1].set_yscale("log")
ax[2].set_yscale("log")

In [ ]:
PyPlot.plot(target[:e], target[:Δe], ".", markersize=1, alpha=0.9)
PyPlot.plot(reference[:e], reference[:Δe], ".", markersize=1, alpha=0.9)
PyPlot.plot(calibtarget[:e], calibtarget[:Δe], ".", markersize=1, alpha=0.9)

In [ ]:
PyPlot.pcolormesh(refmat', norm=PyPlot.matplotlib.colors.LogNorm(),
                  cmap="Reds")
PyPlot.pcolormesh(tarmat', norm=PyPlot.matplotlib.colors.LogNorm(),
                  cmap="Blues")

In [ ]:
refmats = ndimage.gaussian_filter(refmat, sigma=3)
tarmats = ndimage.median_filter(tarmat, 11)
PyPlot.pcolormesh(refmats', norm=PyPlot.matplotlib.colors.LogNorm(),
                  cmap="Reds")
PyPlot.pcolormesh(tarmats', norm=PyPlot.matplotlib.colors.LogNorm(),
                  cmap="Blues")

In [ ]:
Plots.plot()
@df reference Plots.plot(:e, :Δe, markerstrokewidth=0, seriestype=:histogram2d, nbins=1000, 
                 color=:reds)
@df target Plots.plot!(:e, :Δe, markerstrokewidth=0, seriestype=:histogram2d, nbins=1000,
                color=:black)
@df calibtarget Plots.plot!(:e, :Δe, markerstrokewidth=0, seriestype=:histogram2d, nbins=1000,
                color=:greens)

In [ ]:
plt = @df reference Plots.plot(:e, :Δe, markerstrokewidth=0, seriestype=:histogram2d, nbins=1000)

arrays = splitregions2d(refhist.weights, 15, width=nothing, limit=1000);
for array in arrays
    x, y = array.indices
    x = indextoenergy(x, refhist.edges[1])
    y = indextoenergy(y, refhist.edges[2])
    plotbox(((first(x), last(x)), (first(y), last(y))))
    plt = plotbox(array)
end
display(plt)

In [ ]:
ndimage = pyimport("scipy.ndimage")

#refmat = ndimage.gaussian_filter(refhist.weights, sigma=2)
#tarmat = ndimage.gaussian_filter(tarhist.weights, sigma=2)

arrays = splitregions2d(refmat, 15, width=nothing, limit=1000);
feature = arrays[length(arrays)-7]
overlap = featureoverlap2d(feature, tarmat, 40)
i, j = Tuple(argmin(overlap))
plot(log10.(overlap'), seriestype=:contour, alpha=0.9)
scatter!([i, i], [j, j])
#rpx, tpx, rpy, tpy = featurealign2d(refmat, tarmat, 
#    numregions=15, width=nothing, limit=1000, searchwidth=40)

In [ ]:
plt = heatmap(log10transform(refmat'), color=:viridis, size=(800, 800))
arrays = splitregions2d(refmat, 15, width=nothing, limit=1000);
plt = heatmap!(log10transform(tarmat'), color=:plasma)
plotbox.(arrays)
for (rx, tx, ry, ty) in zip(rpx, tpx, rpy, tpy)
    scatter!([rx, tx], [ry, ty], legend=false, markershape=:dtriangle, markerstrokewidth=0,
    markersize=4)
end
plt

In [ ]:
plt = heatmap(log10transform(refhist.weights'), color=:viridis, size=(800, 800))
arrays = splitregions2d(refhist.weights, 15, width=nothing, limit=1000);
plt = heatmap!(log10transform(tarhist.weights'), color=:greens)
plotbox.(arrays)
for (rx, tx, ry, ty) in zip(rpx, tpx, rpy, tpy)
    scatter!([rx, tx], [ry, ty], legend=false, markershape=:dtriangle, markerstrokewidth=0,
    markersize=4)
end
plt

In [ ]:
X = tpx
Y = rpx
x = range(min(X...), max(X...), length=100)
y = @. coeffy[1] + coeffy[2]*x
scatter(X, Y, legend=false)
plot!(x, y)

## Align by projection

In [ ]:
Xref = vec(sum(refmat, dims=2))
Yref = vec(sum(refmat, dims=1))
Xtar = vec(trunc.(Int, sum(tarmat, dims=2)))
Ytar = vec(trunc.(Int, sum(tarmat, dims=1)))
coeff, Xaligned = alignspectra([Yref, Ytar], lowregion=50:600,
    lownumregions=5, lowsearchwidth=100, order=1)
@show coeff

plot(Yref)
plot!(Ytar)
plot!(Xaligned)

## Notes
Method is doomed to fail for low data regions. Reject region if number of counts
is less than N.

Use subarrays for regions.

I think I must apply smoothing to get anything meaningful

Use large regions or small regions? Trade-off between amount of features and neglibability of the higher order terms.

Gainshifting on the indices is impossible. Must instead gainshift on the axis and plot accordingly.

Heatmap is a pure pain, it doesn't take care of the ranges correctly. Use matplotlib pcolormesh? I think so.
Need to take care of all the bookkeeping with index-to-value mapping and the sort when
"gainshifting" the axis.

It is incredibly frustrating to use, save and load OMatrices. Would it be better to instead save the raw events as a long csv of to columns with the E and $\Delta$E? 
 - YES! Wonderful idea. Everything is a thousand times easier now.
  - ~~[ ] Create a plotting recipe for OMatrix~~
  - ~~[ ] Create a gainshift for OMatrix~~
  - Bin the dataframe-data into sizable histograms
  - Featurefit
  - Calibrate the raw data
  - Compare
  - [ ] Fix finding of minimum
  - [ ] Try smoothing
  - Coeffx == coeffy??
  - Idiot! I must use the same binning when binning the data!
  - Feature align on the summed axes instead?